In [1]:
#import semua library yang dibutuhkan
import csv
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
import pandas as pd
df = pd.read_csv('Dataset/BBC News Train.csv')
df

,ArticleId,Text,Category
0,1833,worldcom ex-boss launches defence lawyers defe...,business
1,154,german business confidence slides german busin...,business
2,1101,bbc poll indicates economic gloom citizens in ...,business
3,1976,lifestyle governs mobile choice faster bett...,tech
4,917,enron bosses in $168m payout eighteen former e...,business
...,...,...,...
1485,857,double eviction from big brother model caprice...,entertainment
1486,325,dj double act revamp chart show dj duo jk and ...,entertainment
1487,1590,weak dollar hits reuters revenues at media gro...,business
1488,1587,apple ipod family expands market apple has exp...,tech


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ArticleId  1490 non-null   int64 
 1   Text       1490 non-null   object
 2   Category   1490 non-null   object
dtypes: int64(1), object(2)
memory usage: 35.0+ KB


In [24]:
#Memahami bentuk data
dataset = 'Dataset/BBC News Train.csv'
test_file = 'Dataset/BBC News Test.csv'
with open(dataset) as csvfile:
    print('Berikut adalah nama-nama kolom:\n\n', csvfile.readline())
    print('konten file adalah sebagai berikut:\n\n', csvfile.readline())

Berikut adalah nama-nama kolom:

 ArticleId,Text,Category

konten file adalah sebagai berikut:

 1833,worldcom ex-boss launches defence lawyers defending former worldcom chief bernie ebbers against a battery of fraud charges have called a company whistleblower as their first witness.  cynthia cooper  worldcom s ex-head of internal accounting  alerted directors to irregular accounting practices at the us telecoms giant in 2002. her warnings led to the collapse of the firm following the discovery of an $11bn (Â£5.7bn) accounting fraud. mr ebbers has pleaded not guilty to charges of fraud and conspiracy.  prosecution lawyers have argued that mr ebbers orchestrated a series of accounting tricks at worldcom  ordering employees to hide expenses and inflate revenues to meet wall street earnings estimates. but ms cooper  who now runs her own consulting business  told a jury in new york on wednesday that external auditors arthur andersen had approved worldcom s accounting in early 2001 and 2002

In [5]:
#Buat fungsi untuk membuang semua stopping words, yaitu kata2 yang tidak memiliki makna berarti (general)
def stopwords_remove(sentence):
    stopwords = ["s", "mr", "ms", "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]
    sentences = []
    for word in sentence.lower().split():
        if word.strip('.') in stopwords:
            pass
        else:
            sentences.append(word.strip('.'))
    sentences = ' '.join(sentences)
    return sentences   
        

In [6]:
#Buang semua stopping words, yaitu kata2 yang tidak memiliki makna berarti (general)
#dan ambil kolom berita('Text'), dan kategorinya ('Category')
def parsing_file(file):
    sentences = []
    labels = []
    
    with open(file) as csvfile:
        next(csvfile)
        csvfile = csv.reader(csvfile, delimiter=',')
        for row in csvfile:
            row[1] = stopwords_remove(row[1])
            sentences.append(row[1])
            labels.append(row[2])
        
    return sentences, labels

In [25]:
dataset_sentences, dataset_labels = parsing_file(dataset)
len(dataset_sentences)

1490

In [21]:
#Preprocess the test data
test_id = []
test_sentences = []

with open(test_file) as csvfile:
        next(csvfile)
        csvfile = csv.reader(csvfile, delimiter=',')
        for row in csvfile:
            row[1] = stopwords_remove(row[1])
            test_id.append(row[0])
            test_sentences.append(row[1])

In [23]:
len(test_sentences)
len(test_id)

735

In [ ]:
# len(train_sentences[1].split())
# train_labels[:5]
# len(train_sentences)

## Tokenizer

In [13]:
#Build function for tokenize sentences
def fit_tokenizer(sentences):
    #parameter
    oov_token = '<OOV>'
    vocab_size = 10000
    
    #inisiate the tokenizer class
    tokenizer = Tokenizer(oov_token=oov_token, num_words=vocab_size)
    #fit tokenizer to text
    tokenizer.fit_on_texts(sentences)
    
    return tokenizer

In [26]:
dataset_token = fit_tokenizer(data_sentences)

In [28]:
dataset_word_index = dataset_token.word_index

In [ ]:
# len(train_word_index)

## Splitting data

In [32]:
#data splitting
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset_sentences,
                                                      dataset_labels,
                                                      test_size=0.3,
                                                      stratify = dataset_labels,
                                                      random_state=0)

In [31]:
def generate_pad_seq(tokenizer, sentences):
    #parameter
    maxlen = 1500
    truncating = 'post'
    
    #algoritma
    sequences = tokenizer.texts_to_sequences(sentences)
    padded = pad_sequences(sequences, maxlen=maxlen, truncating=truncating)
    
    return padded

In [34]:
X_train = generate_pad_seq(dataset_token, X_train)
X_test = generate_pad_seq(dataset_token, X_test)

In [35]:
len(X_train) + len(X_test)

1490

In [52]:
#Encode the category label
from sklearn.preprocessing import LabelEncoder
import numpy as np
import tensorflow as tf

le = LabelEncoder()

train_labels = le.fit_transform(y_train)
train_labels = np.asarray(tf.keras.utils.to_categorical(train_labels))
test_labels = le.fit_transform(y_test)
test_labels = np.asarray(tf.keras.utils.to_categorical(test_labels))
# test_labels
# train_labels

In [53]:
#Build tensorflow dataset from train and test data
train_ds = tf.data.Dataset.from_tensor_slices((X_train,train_labels))
test_ds = tf.data.Dataset.from_tensor_slices((X_test,test_labels))

In [56]:
#Mengecek bagaimana bentuk datanya
count =0
print('======Train dataset ====')
for value,label in train_ds:
    count += 1
    print('Sentence:', value, '\nLabel:', label)
    if count==3:
        break
count =0

======Train dataset ====
Sentence: tf.Tensor([   0    0    0 ... 1107  802 4067], shape=(1500,), dtype=int32) 
Label: tf.Tensor([0. 0. 0. 1. 0.], shape=(5,), dtype=float32)
Sentence: tf.Tensor([   0    0    0 ... 2456    1  701], shape=(1500,), dtype=int32) 
Label: tf.Tensor([1. 0. 0. 0. 0.], shape=(5,), dtype=float32)
Sentence: tf.Tensor([   0    0    0 ... 1422  313 1729], shape=(1500,), dtype=int32) 
Label: tf.Tensor([0. 0. 1. 0. 0.], shape=(5,), dtype=float32)


In [ ]:
# train_sequences, train_padded = generate_pad_seq(train_token, train_sentences)

In [ ]:
# test_sequences, tes_padded = generate_pad_seq(train_token, test_sentences)

In [ ]:
# train_sequences[0]
# len(train_sequences[0])
# len(train_sentences[0].split())
# len(train_padded[0])
